# Implementación del Análisis de sentimiento.

## Importación de librerías y configuraciones iniciales

In [1]:

from pathlib import Path
from typing import List, Tuple, Dict, Optional
import re
import numpy as np
import pandas as pd
from collections import defaultdict

# Sklearn imports
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report
from sklearn.base import BaseEstimator, TransformerMixin

print("✅ Imports cargados correctamente")

✅ Imports cargados correctamente


## Definición de funciones para carga de datos y transformación

El flujo de procesamiento comienza con la transformación de cada línea de reseña en una representación estructurada. A partir de cadenas de texto que contienen tokens acompañados de sus frecuencias, se construye un diccionario de características acumuladas y, cuando está disponible, se extrae la etiqueta de sentimiento en formato binario.

In [2]:
def parse_review_counts_line(line: str) -> Tuple[Dict[str, float], Optional[int]]:
    """
    Devuelve (features_dict, label) donde label es 1=positive, 0=negative o None si no viene.
    Ignora pares malformados.
    """
    feats = {}
    y = None
    parts = line.strip().split()
    for item in parts:
        if item.startswith("#label#:"):
            raw = item.split(":", 1)[1].strip().lower()
            if raw in ("positive", "pos", "1", "p"):
                y = 1
            elif raw in ("negative", "neg", "0", "n"):
                y = 0
            continue
        if ":" not in item:
            continue
        tok, cnt = item.split(":", 1)
        try:
            val = int(cnt)
        except ValueError:
            try:
                val = float(cnt)
            except ValueError:
                continue
        if tok in feats:
            feats[tok] = feats[tok] + val
        else:
            feats[tok] = val
    return feats, y

Con este mecanismo establecido, se habilita la lectura sistemática de archivos de reseñas. Cada archivo es procesado línea por línea, generando colecciones de ejemplos donde las características textuales quedan vinculadas a etiquetas de clase, o en su defecto se asigna una etiqueta predeterminada. Esto permite integrar en un mismo esquema tanto datos etiquetados como no etiquetados, facilitando posteriores etapas de análisis.

In [3]:
def load_review_file(path: Path, default_label: Optional[int] = None):
    """
    Lee un .review línea por línea y devuelve (X_dicts, y_labels).
    - Si la línea trae #label#, se usa.
    - Si no trae y default_label no es None, se asigna ese.
    """
    X, y = [], []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if line == "":
                continue
            feats, lbl = parse_review_counts_line(line)
            if lbl is None:
                lbl = default_label
            if feats:
                X.append(feats)
                y.append(lbl)
    return X, y

A nivel organizacional, los datos se estructuran de dos formas complementarias. Por un lado, es posible cargar la información categoría por categoría, diferenciando reseñas positivas, negativas y no etiquetadas, lo cual permite realizar un estudio detallado de cada dominio de interés. Por otro, también se ofrece la combinación de todas las categorías en un único conjunto global, en el que se reúnen reseñas etiquetadas y no etiquetadas de manera conjunta.

In [4]:
def load_all_domains_by_category(root: Path, categories: List[str]) -> Dict[str, Dict[str, List]]:
    """
    Carga datos organizados por categoría para análisis individual.
    Retorna: {categoria: {X_pos, y_pos, X_neg, y_neg, X_unl}}
    """
    data = {}
    for cat in categories:
        d = root / cat
        Xp, yp = load_review_file(d / "positive.review", default_label=1)
        Xn, yn = load_review_file(d / "negative.review", default_label=0)
        Xu, yu = load_review_file(d / "unlabeled.review", default_label=None)
        data[cat] = {
            "X_pos": Xp, "y_pos": yp,
            "X_neg": Xn, "y_neg": yn,
            "X_unl": Xu
        }
        print(f"[{cat}] pos={len(Xp)} neg={len(Xn)} unl={len(Xu)}")
    return data

In [5]:
def load_all_domains_combined(root: Path, categories: List[str]) -> Dict[str, List]:
    """
    Carga datos combinados de todas las categorías para análisis global.
    Retorna: {X_labeled, y_labeled, X_unlabeled}
    """
    X_lab, y_lab = [], []
    X_unl = []

    for cat in categories:
        d = root / cat
        Xp, yp = load_review_file(d / "positive.review", default_label=1)
        Xn, yn = load_review_file(d / "negative.review", default_label=0)
        Xu, yu = load_review_file(d / "unlabeled.review", default_label=None)

        X_lab.extend(Xp)
        y_lab.extend(yp)
        X_lab.extend(Xn)
        y_lab.extend(yn)
        X_unl.extend(Xu)

    return {
        "X_labeled": X_lab,
        "y_labeled": y_lab,
        "X_unlabeled": X_unl
    }

Finalmente, el flujo incorpora un recurso léxico externo a través de la integración de SentiWordNet. A partir de este repositorio, se construye un diccionario donde cada palabra queda asociada a su puntaje medio de positividad y negatividad, calculado sobre sus distintas apariciones.

In [6]:
def load_sentiwordnet(path: Path) -> Dict[str, Tuple[float, float]]:
    """
    Retorna dict: palabra -> (pos_score_promedio, neg_score_promedio)
    Ignora líneas de comentario que comienzan con '#'.
    """
    pos_acc = defaultdict(float)
    neg_acc = defaultdict(float)
    cnt_acc = defaultdict(int)

    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            if not line or line.startswith("#"):
                continue
            cols = line.strip().split("\t")
            if len(cols) < 5:
                continue
            try:
                pos_score = float(cols[2])
                neg_score = float(cols[3])
            except ValueError:
                continue
            terms = cols[4].split()
            for t in terms:
                lemma = t.split("#", 1)[0].lower()
                pos_acc[lemma] += pos_score
                neg_acc[lemma] += neg_score
                cnt_acc[lemma] += 1

    lex = {}
    for w in cnt_acc:
        lex[w] = (pos_acc[w] / cnt_acc[w], neg_acc[w] / cnt_acc[w])
    return lex

print("✅ Funciones de carga de datos definidas")

✅ Funciones de carga de datos definidas


Sobre la base de los datos cargados y el léxico construido, se incorporan ahora mecanismos para transformar la información en representaciones numéricas más ricas. Se diseñan atributos derivados de SentiWordNet que resumen distintos aspectos del tono emocional de los textos: desde la suma ponderada de puntajes positivos y negativos hasta medidas de frecuencia relativa, máximos observados y promedios entre los términos reconocidos.

In [7]:
class SentiWordNetFeatures(BaseEstimator, TransformerMixin):
    def __init__(self, lexicon: Dict[str, Tuple[float, float]]):
        self.lexicon = lexicon

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        rows = []
        for feats in X:  # feats es un dict token -> count
            total_tokens = len(feats)
            sum_pos = 0.0
            sum_neg = 0.0
            hits_pos = 0.0
            hits_neg = 0.0
            max_pos = 0.0
            max_neg = 0.0
            acc_pos = 0.0
            acc_neg = 0.0
            seen = 0.0

            for tok, cnt in feats.items():
                key = tok.lower()
                if key in self.lexicon:
                    p, n = self.lexicon[key]
                    sum_pos = sum_pos + p * cnt
                    sum_neg = sum_neg + n * cnt
                    acc_pos = acc_pos + p
                    acc_neg = acc_neg + n
                    seen = seen + 1.0
                    if p > 0:
                        hits_pos = hits_pos + 1.0
                        if p > max_pos:
                            max_pos = p
                    if n > 0:
                        hits_neg = hits_neg + 1.0
                        if n > max_neg:
                            max_neg = n

            if total_tokens > 0:
                ratio_pos = hits_pos / float(total_tokens)
                ratio_neg = hits_neg / float(total_tokens)
            else:
                ratio_pos = 0.0
                ratio_neg = 0.0

            if seen > 0:
                avg_pos = acc_pos / seen
                avg_neg = acc_neg / seen
            else:
                avg_pos = 0.0
                avg_neg = 0.0

            rows.append([sum_pos, sum_neg, hits_pos, hits_neg,
                         ratio_pos, ratio_neg, max_pos, max_neg,
                         avg_pos, avg_neg])

        arr = np.array(rows, dtype=float)
        return arr 

Con este insumo, se preparan diferentes configuraciones de modelos de referencia que permiten comparar alternativas de representación y clasificación. Entre ellas se consideran esquemas basados en Naive Bayes y Regresión Logística, aplicados tanto sobre recuentos simples de términos como sobre transformaciones TF-IDF que ponderan la relevancia de las palabras en el corpus.

In [8]:
def make_nb_tf():
    return Pipeline([
        ("dict", DictVectorizer(sparse=True)),
        ("nb", MultinomialNB())
    ])

In [9]:
def make_nb_tfidf():
    return Pipeline([
        ("dict", DictVectorizer(sparse=True)),
        ("tfidf", TfidfTransformer()),
        ("nb", MultinomialNB())
    ])

In [10]:
def make_lr_tf():
    clf = LogisticRegression(
        solver="liblinear", multi_class="ovr", penalty="l2",
        C=1.0, max_iter=2000, tol=1e-3, n_jobs=1, random_state=42
    )
    return Pipeline([
        ("dict", DictVectorizer(sparse=True)),
        ("lr", clf)
    ])

In [11]:
def make_lr_tfidf():
    clf = LogisticRegression(
        solver="liblinear", multi_class="ovr", penalty="l2",
        C=1.0, max_iter=2000, tol=1e-3, n_jobs=1, random_state=42
    )
    return Pipeline([
        ("dict", DictVectorizer(sparse=True)),
        ("tfidf", TfidfTransformer()),
        ("lr", clf)
    ])

En paralelo, se contemplan configuraciones que aprovechan directamente las características derivadas del léxico. En este caso, los vectores sentimentales generados se normalizan y se introducen en clasificadores como Regresión Logística o Gaussian Naive Bayes.

In [12]:
def make_lr_lexicon(lexicon: Dict[str, Tuple[float, float]]):
    clf = LogisticRegression(
        solver="liblinear", multi_class="ovr", penalty="l2",
        C=1.0, max_iter=2000, tol=1e-3, n_jobs=1, random_state=42
    )
    return Pipeline([
        ("lex", SentiWordNetFeatures(lexicon)),
        ("sc", StandardScaler(with_mean=False)),
        ("lr", clf)
    ])

In [13]:
def make_nb_lexicon(lexicon: Dict[str, Tuple[float, float]]):
    # Usar GaussianNB para características continuas del léxico
    clf = GaussianNB()
    return Pipeline([
        ("lex", SentiWordNetFeatures(lexicon)),
        ("sc", StandardScaler(with_mean=False)),
        ("nb", clf)
    ])

## Ejecución del Pipeline

In [14]:
# Rutas (AJUSTAR SEGÚN TU SISTEMA)
ROOT = Path(r"Datasets/Multi Domain Sentiment/processed_acl/processed_acl")
CATEGORIES = ["Books", "DVD", "Electronics", "Kitchen"]
SWN_PATH = Path(r"Datasets/EN_Lexicons/SentiWordNet_3.0.0.txt")

In [15]:
print("Cargando datos por categoría...")
data_by_category = load_all_domains_by_category(ROOT, CATEGORIES)

Cargando datos por categoría...
[Books] pos=1000 neg=1000 unl=4465
[DVD] pos=1000 neg=1000 unl=3586
[Electronics] pos=1000 neg=1000 unl=5681
[Kitchen] pos=1000 neg=1000 unl=5945


In [16]:
print("\nCargando léxico SentiWordNet...")
lexicon = load_sentiwordnet(SWN_PATH)
print(f"Léxico cargado: {len(lexicon)} palabras")


Cargando léxico SentiWordNet...
Léxico cargado: 147306 palabras


## Evaluación de los modelos individuales

In [17]:
def evaluar_modelo(nombre, pipe, X_tr, y_tr, X_te, y_te, mostrar_reporte=False):
    """Evalúa un modelo y retorna métricas"""
    pipe.fit(X_tr, y_tr)
    y_pred = pipe.predict(X_te)
    
    p_mac, r_mac, f1_mac, _ = precision_recall_fscore_support(y_te, y_pred, average="macro", zero_division=0)
    p_mic, r_mic, f1_mic, _ = precision_recall_fscore_support(y_te, y_pred, average="micro", zero_division=0)
    acc = accuracy_score(y_te, y_pred)
    
    if mostrar_reporte:
        print(f"\n{nombre}")
        print("Accuracy: {:.4f}".format(acc))
        print("Macro  -> P: {:.4f}  R: {:.4f}  F1: {:.4f}".format(p_mac, r_mac, f1_mac))
        print("Micro  -> P: {:.4f}  R: {:.4f}  F1: {:.4f}".format(p_mic, r_mic, f1_mic))
        print("\nReporte por clase:")
        print(classification_report(y_te, y_pred, target_names=["negative", "positive"], digits=3, zero_division=0))
    
    return {
        "Accuracy": acc,
        "F1 Macro": f1_mac,
        "F1 Micro": f1_mic,
        "Precision Macro": p_mac,
        "Recall Macro": r_mac
    }

In [ ]:
def analizar_categoria(categoria, data_cat, lexicon):
    """Analiza una categoría específica con los 6 modelos"""
    print(f"\n{'='*100}")
    print(f"ANÁLISIS CATEGORÍA: {categoria.upper()}")
    print(f"{'='*100}")
    
    # Preparar datos
    X_lab = data_cat["X_pos"] + data_cat["X_neg"]
    y_lab = data_cat["y_pos"] + data_cat["y_neg"]
    
    X_tr, X_te, y_tr, y_te = train_test_split(
        X_lab, y_lab, test_size=0.30, random_state=42, stratify=y_lab
    )
    
    # Definir los 6 modelos
    modelos = {
        "NB + TF": make_nb_tf(),
        "NB + TF-IDF": make_nb_tfidf(),
        "LR + TF": make_lr_tf(),
        "LR + TF-IDF": make_lr_tfidf(),
        "LR + Lexicon(SWN)": make_lr_lexicon(lexicon),
        "NB + Lexicon(SWN)": make_nb_lexicon(lexicon)
    }
    
    # Evaluar cada modelo
    resultados = {}
    modelos_entrenados = {}
    
    for nombre, pipe in modelos.items():
        print(f"\nEntrenando {nombre}...")
        try:
            resultado = evaluar_modelo(f"[{categoria}] {nombre}", pipe, X_tr, y_tr, X_te, y_te, mostrar_reporte=True)
            resultados[nombre] = resultado
            modelos_entrenados[nombre] = pipe
        except Exception as e:
            print(f"Error con {nombre}: {e}")
            continue
    
    # Crear tabla resumen
    df_resultados = pd.DataFrame(resultados).T
    print(f"\n{'='*60}")
    print(f"RESUMEN {categoria}")
    print(f"{'='*60}")
    print(df_resultados[["Accuracy", "F1 Macro", "F1 Micro"]].round(4))
    
    # Encontrar mejor modelo y predecir unlabeled
    if resultados:
        mejor_nombre = max(resultados.keys(), key=lambda x: resultados[x]["F1 Macro"])
        X_unl = data_cat["X_unl"]
        if X_unl:
            y_pred_unl = modelos_entrenados[mejor_nombre].predict(X_unl)
            print(f"\nMejor modelo: {mejor_nombre}")
            print(f"Predicciones unlabeled (primeros 10): {list(y_pred_unl[:10])}")
            print(f"Distribución unlabeled: Pos={np.sum(y_pred_unl)} | Neg={len(y_pred_unl)-np.sum(y_pred_unl)}")
    
    return df_resultados, modelos_entrenados

### Evaluación de resultados en una categoría específica

In [19]:
# Ejecutar análisis por categoría
tablas_categoria = {}
modelos_categoria = {}

for categoria in CATEGORIES:
    df_cat, models_cat = analizar_categoria(categoria, data_by_category[categoria], lexicon)
    tablas_categoria[categoria] = df_cat
    modelos_categoria[categoria] = models_cat


ANÁLISIS CATEGORÍA: BOOKS

Entrenando NB + TF...

[Books] NB + TF
Accuracy: 0.7650
Macro  -> P: 0.7837  R: 0.7650  F1: 0.7611
Micro  -> P: 0.7650  R: 0.7650  F1: 0.7650

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.711     0.893     0.792       300
    positive      0.857     0.637     0.730       300

    accuracy                          0.765       600
   macro avg      0.784     0.765     0.761       600
weighted avg      0.784     0.765     0.761       600


Entrenando NB + TF-IDF...

[Books] NB + TF-IDF
Accuracy: 0.7883
Macro  -> P: 0.8162  R: 0.7883  F1: 0.7836
Micro  -> P: 0.7883  R: 0.7883  F1: 0.7883

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.722     0.937     0.816       300
    positive      0.910     0.640     0.751       300

    accuracy                          0.788       600
   macro avg      0.816     0.788     0.784       600
weighted avg      0.816     0.788     0.

c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



[Books] LR + TF
Accuracy: 0.8067
Macro  -> P: 0.8078  R: 0.8067  F1: 0.8065
Micro  -> P: 0.8067  R: 0.8067  F1: 0.8067

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.789     0.837     0.812       300
    positive      0.826     0.777     0.801       300

    accuracy                          0.807       600
   macro avg      0.808     0.807     0.806       600
weighted avg      0.808     0.807     0.806       600


Entrenando LR + TF-IDF...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



[Books] LR + TF-IDF
Accuracy: 0.8033
Macro  -> P: 0.8106  R: 0.8033  F1: 0.8022
Micro  -> P: 0.8033  R: 0.8033  F1: 0.8033

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.763     0.880     0.817       300
    positive      0.858     0.727     0.787       300

    accuracy                          0.803       600
   macro avg      0.811     0.803     0.802       600
weighted avg      0.811     0.803     0.802       600


Entrenando LR + Lexicon(SWN)...

[Books] LR + Lexicon(SWN)
Accuracy: 0.7100
Macro  -> P: 0.7105  R: 0.7100  F1: 0.7098
Micro  -> P: 0.7100  R: 0.7100  F1: 0.7100

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.701     0.733     0.717       300
    positive      0.720     0.687     0.703       300

    accuracy                          0.710       600
   macro avg      0.710     0.710     0.710       600
weighted avg      0.710     0.710     0.710       600


Entrenando NB + Le

...


[Electronics] LR + TF-IDF
Accuracy: 0.8300
Macro  -> P: 0.8302  R: 0.8300  F1: 0.8300
Micro  -> P: 0.8300  R: 0.8300  F1: 0.8300

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.839     0.817     0.828       300
    positive      0.821     0.843     0.832       300

    accuracy                          0.830       600
   macro avg      0.830     0.830     0.830       600
weighted avg      0.830     0.830     0.830       600


Entrenando LR + Lexicon(SWN)...

[Electronics] LR + Lexicon(SWN)
Accuracy: 0.7450
Macro  -> P: 0.7470  R: 0.7450  F1: 0.7445
Micro  -> P: 0.7450  R: 0.7450  F1: 0.7450

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.725     0.790     0.756       300
    positive      0.769     0.700     0.733       300

    accuracy                          0.745       600
   macro avg      0.747     0.745     0.744       600
weighted avg      0.747     0.745     0.744       600


Entren

c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



[Kitchen] LR + TF
Accuracy: 0.8850
Macro  -> P: 0.8850  R: 0.8850  F1: 0.8850
Micro  -> P: 0.8850  R: 0.8850  F1: 0.8850

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.884     0.887     0.885       300
    positive      0.886     0.883     0.885       300

    accuracy                          0.885       600
   macro avg      0.885     0.885     0.885       600
weighted avg      0.885     0.885     0.885       600


Entrenando LR + TF-IDF...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



[Kitchen] LR + TF-IDF
Accuracy: 0.8833
Macro  -> P: 0.8834  R: 0.8833  F1: 0.8833
Micro  -> P: 0.8833  R: 0.8833  F1: 0.8833

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.889     0.877     0.883       300
    positive      0.878     0.890     0.884       300

    accuracy                          0.883       600
   macro avg      0.883     0.883     0.883       600
weighted avg      0.883     0.883     0.883       600


Entrenando LR + Lexicon(SWN)...

[Kitchen] LR + Lexicon(SWN)
Accuracy: 0.7417
Macro  -> P: 0.7417  R: 0.7417  F1: 0.7417
Micro  -> P: 0.7417  R: 0.7417  F1: 0.7417

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.739     0.747     0.743       300
    positive      0.744     0.737     0.740       300

    accuracy                          0.742       600
   macro avg      0.742     0.742     0.742       600
weighted avg      0.742     0.742     0.742       600


Entrenando NB 

### Evaluación de resultados TF vs TF-IDF vs Lexicons

In [28]:
def comparar_por_tipo_representacion(tablas_categoria, categorias):
    """
    Compara el rendimiento promedio por tipo de representación
    """
    print("="*80)
    print("COMPARACIÓN POR TIPO DE REPRESENTACIÓN DE CARACTERÍSTICAS")
    print("="*80)
    
    # Agrupar modelos por tipo de representación
    tipos_representacion = {
        'TF (Term Frequency)': ['NB + TF', 'LR + TF'],
        'TF-IDF': ['NB + TF-IDF', 'LR + TF-IDF'], 
        'Lexicon Features': ['LR + Lexicon(SWN)', 'NB + Lexicon(SWN)']
    }
    
    resultados_por_tipo = {}
    
    for tipo, modelos in tipos_representacion.items():
        f1_scores = []
        accuracy_scores = []
        
        for categoria in categorias:
            if categoria in tablas_categoria:
                tabla = tablas_categoria[categoria]
                for modelo in modelos:
                    if modelo in tabla.index:
                        f1_scores.append(tabla.loc[modelo, 'F1 Macro'])
                        accuracy_scores.append(tabla.loc[modelo, 'Accuracy'])
        
        if f1_scores:
            resultados_por_tipo[tipo] = {
                'F1 Macro Promedio': np.mean(f1_scores),
                'F1 Macro Std': np.std(f1_scores),
                'Accuracy Promedio': np.mean(accuracy_scores),
                'Accuracy Std': np.std(accuracy_scores),
                'Mejor F1': np.max(f1_scores),
                'Peor F1': np.min(f1_scores)
            }
    
    # Crear DataFrame para comparación
    df_tipos = pd.DataFrame(resultados_por_tipo).T
    df_tipos = df_tipos.sort_values('F1 Macro Promedio', ascending=False)
    
    print("\nRENDIMIENTO PROMEDIO POR TIPO DE REPRESENTACIÓN:")
    print("-" * 60)
    print(df_tipos.round(4))
    
    # Análisis de estabilidad
    print("\n\nANÁLISIS DE ESTABILIDAD (Desviación Estándar):")
    print("-" * 50)
    estabilidad = df_tipos[['F1 Macro Std', 'Accuracy Std']].sort_values('F1 Macro Std')
    print("Representación más estable (menor std):")
    print(estabilidad.round(4))
    
    return df_tipos

def comparar_por_categoria_representacion(tablas_categoria, categorias):
    """
    Analiza qué representación funciona mejor en cada categoría
    """
    print("\n" + "="*80)
    print("MEJOR REPRESENTACIÓN POR CATEGORÍA")
    print("="*80)
    
    mejor_por_categoria = {}
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            # Agrupar por tipo de representación
            tf_models = [modelo for modelo in tabla.index if 'TF' in modelo and 'TF-IDF' not in modelo]
            tfidf_models = [modelo for modelo in tabla.index if 'TF-IDF' in modelo]
            lexicon_models = [modelo for modelo in tabla.index if 'Lexicon' in modelo]
            
            resultados_categoria = {}
            
            # Mejor modelo por tipo
            if tf_models:
                best_tf = tabla.loc[tf_models, 'F1 Macro'].max()
                best_tf_model = tabla.loc[tf_models, 'F1 Macro'].idxmax()
                resultados_categoria['TF'] = {'F1': best_tf, 'Modelo': best_tf_model}
            
            if tfidf_models:
                best_tfidf = tabla.loc[tfidf_models, 'F1 Macro'].max()
                best_tfidf_model = tabla.loc[tfidf_models, 'F1 Macro'].idxmax()
                resultados_categoria['TF-IDF'] = {'F1': best_tfidf, 'Modelo': best_tfidf_model}
            
            if lexicon_models:
                best_lexicon = tabla.loc[lexicon_models, 'F1 Macro'].max()
                best_lexicon_model = tabla.loc[lexicon_models, 'F1 Macro'].idxmax()
                resultados_categoria['Lexicon'] = {'F1': best_lexicon, 'Modelo': best_lexicon_model}
            
            mejor_por_categoria[categoria] = resultados_categoria
            
            print(f"\n{categoria}:")
            print("-" * 40)
            for repr_type, data in resultados_categoria.items():
                print(f"  {repr_type:12s}: F1={data['F1']:.4f} ({data['Modelo']})")
            
            # Identificar la mejor representación para esta categoría
            if resultados_categoria:
                mejor_repr = max(resultados_categoria.keys(), 
                               key=lambda x: resultados_categoria[x]['F1'])
                print(f"  → MEJOR: {mejor_repr} (F1={resultados_categoria[mejor_repr]['F1']:.4f})")
    
    return mejor_por_categoria

def analizar_complementariedad_representaciones(tablas_categoria, categorias):
    """
    Analiza si diferentes representaciones capturan aspectos complementarios
    """
    print("\n" + "="*80)
    print("ANÁLISIS DE COMPLEMENTARIEDAD ENTRE REPRESENTACIONES")
    print("="*80)
    
    # Calcular correlaciones entre tipos de representación
    correlaciones = {}
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            # Extraer F1 scores por tipo
            tf_scores = []
            tfidf_scores = []
            lexicon_scores = []
            
            modelos_tf = [m for m in tabla.index if 'TF' in m and 'TF-IDF' not in m]
            modelos_tfidf = [m for m in tabla.index if 'TF-IDF' in m]
            modelos_lexicon = [m for m in tabla.index if 'Lexicon' in m]
            
            if modelos_tf:
                tf_scores.extend(tabla.loc[modelos_tf, 'F1 Macro'].tolist())
            if modelos_tfidf:
                tfidf_scores.extend(tabla.loc[modelos_tfidf, 'F1 Macro'].tolist())
            if modelos_lexicon:
                lexicon_scores.extend(tabla.loc[modelos_lexicon, 'F1 Macro'].tolist())
    
    print("\nRESUMEN DE OBSERVACIONES:")
    print("-" * 50)
    
    # Análisis de brechas de rendimiento
    print("\n1. BRECHAS DE RENDIMIENTO:")
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            tf_best = tabla.loc[[m for m in tabla.index if 'TF' in m and 'TF-IDF' not in m], 'F1 Macro'].max() if any('TF' in m and 'TF-IDF' not in m for m in tabla.index) else 0
            tfidf_best = tabla.loc[[m for m in tabla.index if 'TF-IDF' in m], 'F1 Macro'].max() if any('TF-IDF' in m for m in tabla.index) else 0
            lexicon_best = tabla.loc[[m for m in tabla.index if 'Lexicon' in m], 'F1 Macro'].max() if any('Lexicon' in m for m in tabla.index) else 0
            
            tfidf_vs_tf = tfidf_best - tf_best
            lexicon_vs_tfidf = lexicon_best - tfidf_best
            
            print(f"  {categoria:12s}: TF-IDF vs TF = {tfidf_vs_tf:+.4f}, Lexicon vs TF-IDF = {lexicon_vs_tfidf:+.4f}")
    
    print("\n2. INTERPRETACIÓN:")
    print("  • TF-IDF > TF: TF-IDF normaliza y reduce ruido de palabras muy frecuentes")
    print("  • Lexicon vs TF-IDF: Características semánticas vs estadísticas")
    print("  • Variación entre categorías indica especificidad de dominio")

# Función principal para ejecutar todas las comparaciones
def ejecutar_comparacion_completa(tablas_categoria, categorias):
    """
    Ejecuta todas las comparaciones de representación de características
    """
    # 1. Comparación general por tipo
    df_tipos = comparar_por_tipo_representacion(tablas_categoria, categorias)
    
    # 2. Mejor representación por categoría  
    mejor_por_cat = comparar_por_categoria_representacion(tablas_categoria, categorias)
    
    # 3. Análisis de complementariedad
    analizar_complementariedad_representaciones(tablas_categoria, categorias)
    
    return df_tipos, mejor_por_cat

# uso:
df_tipos, mejor_por_cat = ejecutar_comparacion_completa(tablas_categoria, CATEGORIES)

COMPARACIÓN POR TIPO DE REPRESENTACIÓN DE CARACTERÍSTICAS

RENDIMIENTO PROMEDIO POR TIPO DE REPRESENTACIÓN:
------------------------------------------------------------
                     F1 Macro Promedio  F1 Macro Std  Accuracy Promedio  \
TF-IDF                          0.8396        0.0322             0.8404   
TF (Term Frequency)             0.8317        0.0363             0.8323   
Lexicon Features                0.7041        0.0346             0.7056   

                     Accuracy Std  Mejor F1  Peor F1  
TF-IDF                     0.0311    0.8833   0.7836  
TF (Term Frequency)        0.0354    0.8850   0.7611  
Lexicon Features           0.0339    0.7445   0.6460  


ANÁLISIS DE ESTABILIDAD (Desviación Estándar):
--------------------------------------------------
Representación más estable (menor std):
                     F1 Macro Std  Accuracy Std
TF-IDF                     0.0322        0.0311
Lexicon Features           0.0346        0.0339
TF (Term Frequency)       

In [29]:
# =============================================================================
# COMPARACIÓN DE ALGORITMOS: NAIVE BAYES vs LOGISTIC REGRESSION
# =============================================================================

def comparar_nb_vs_lr(tablas_categoria, categorias):
    """
    Compara el rendimiento promedio entre Naive Bayes y Logistic Regression
    """
    print("="*80)
    print("COMPARACIÓN DE ALGORITMOS: NAIVE BAYES vs LOGISTIC REGRESSION")
    print("="*80)
    
    # Agrupar modelos por algoritmo
    algoritmos = {
        'Naive Bayes': [modelo for modelo in ['NB + TF', 'NB + TF-IDF', 'NB + Lexicon(SWN)']],
        'Logistic Regression': [modelo for modelo in ['LR + TF', 'LR + TF-IDF', 'LR + Lexicon(SWN)']]
    }
    
    resultados_por_algoritmo = {}
    
    for algoritmo, modelos in algoritmos.items():
        f1_scores = []
        accuracy_scores = []
        precision_scores = []
        recall_scores = []
        
        for categoria in categorias:
            if categoria in tablas_categoria:
                tabla = tablas_categoria[categoria]
                for modelo in modelos:
                    if modelo in tabla.index:
                        f1_scores.append(tabla.loc[modelo, 'F1 Macro'])
                        accuracy_scores.append(tabla.loc[modelo, 'Accuracy'])
                        precision_scores.append(tabla.loc[modelo, 'Precision Macro'])
                        recall_scores.append(tabla.loc[modelo, 'Recall Macro'])
        
        if f1_scores:
            resultados_por_algoritmo[algoritmo] = {
                'F1 Macro Promedio': np.mean(f1_scores),
                'F1 Macro Std': np.std(f1_scores),
                'Accuracy Promedio': np.mean(accuracy_scores),
                'Accuracy Std': np.std(accuracy_scores),
                'Precision Promedio': np.mean(precision_scores),
                'Recall Promedio': np.mean(recall_scores),
                'Mejor F1': np.max(f1_scores),
                'Peor F1': np.min(f1_scores),
                'Num Modelos': len(f1_scores)
            }
    
    # Crear DataFrame para comparación
    df_algoritmos = pd.DataFrame(resultados_por_algoritmo).T
    df_algoritmos = df_algoritmos.sort_values('F1 Macro Promedio', ascending=False)
    
    print("\nRENDIMIENTO PROMEDIO POR ALGORITMO:")
    print("-" * 60)
    print(df_algoritmos[['F1 Macro Promedio', 'F1 Macro Std', 'Accuracy Promedio', 'Accuracy Std']].round(4))
    
    # Análisis de ventaja estadística
    nb_scores = []
    lr_scores = []
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            # Recopilar todos los scores de NB y LR
            for modelo in tabla.index:
                if 'NB' in modelo:
                    nb_scores.append(tabla.loc[modelo, 'F1 Macro'])
                elif 'LR' in modelo:
                    lr_scores.append(tabla.loc[modelo, 'F1 Macro'])
    
    diferencia_promedio = np.mean(lr_scores) - np.mean(nb_scores)
    print(f"\nDIFERENCIA PROMEDIO (LR - NB): {diferencia_promedio:+.4f}")
    
    if diferencia_promedio > 0:
        print("→ Logistic Regression supera a Naive Bayes en promedio")
    else:
        print("→ Naive Bayes supera a Logistic Regression en promedio")
    
    return df_algoritmos

def analizar_nb_vs_lr_por_categoria(tablas_categoria, categorias):
    """
    Analiza qué algoritmo funciona mejor en cada categoría
    """
    print("\n" + "="*80)
    print("ANÁLISIS NB vs LR POR CATEGORÍA")
    print("="*80)
    
    mejor_por_categoria = {}
    ventajas_detalladas = {}
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            # Separar modelos por algoritmo
            nb_models = [modelo for modelo in tabla.index if 'NB' in modelo]
            lr_models = [modelo for modelo in tabla.index if 'LR' in modelo]
            
            resultados_categoria = {}
            
            # Estadísticas por algoritmo en esta categoría
            if nb_models:
                nb_f1_scores = tabla.loc[nb_models, 'F1 Macro']
                best_nb = nb_f1_scores.max()
                best_nb_model = nb_f1_scores.idxmax()
                avg_nb = nb_f1_scores.mean()
                resultados_categoria['NB'] = {
                    'Mejor F1': best_nb, 
                    'Mejor Modelo': best_nb_model,
                    'F1 Promedio': avg_nb,
                    'Todos los F1': nb_f1_scores.tolist()
                }
            
            if lr_models:
                lr_f1_scores = tabla.loc[lr_models, 'F1 Macro']
                best_lr = lr_f1_scores.max()
                best_lr_model = lr_f1_scores.idxmax()
                avg_lr = lr_f1_scores.mean()
                resultados_categoria['LR'] = {
                    'Mejor F1': best_lr,
                    'Mejor Modelo': best_lr_model, 
                    'F1 Promedio': avg_lr,
                    'Todos los F1': lr_f1_scores.tolist()
                }
            
            mejor_por_categoria[categoria] = resultados_categoria
            
            print(f"\n{categoria}:")
            print("-" * 40)
            
            if 'NB' in resultados_categoria and 'LR' in resultados_categoria:
                nb_data = resultados_categoria['NB']
                lr_data = resultados_categoria['LR']
                
                print(f"  Naive Bayes:")
                print(f"    Mejor:    F1={nb_data['Mejor F1']:.4f} ({nb_data['Mejor Modelo']})")
                print(f"    Promedio: F1={nb_data['F1 Promedio']:.4f}")
                
                print(f"  Logistic Regression:")
                print(f"    Mejor:    F1={lr_data['Mejor F1']:.4f} ({lr_data['Mejor Modelo']})")
                print(f"    Promedio: F1={lr_data['F1 Promedio']:.4f}")
                
                # Determinar ganador
                if lr_data['Mejor F1'] > nb_data['Mejor F1']:
                    ventaja = lr_data['Mejor F1'] - nb_data['Mejor F1']
                    ganador = "LR"
                    print(f"  → GANADOR: LR (ventaja: +{ventaja:.4f})")
                elif nb_data['Mejor F1'] > lr_data['Mejor F1']:
                    ventaja = nb_data['Mejor F1'] - lr_data['Mejor F1']
                    ganador = "NB"
                    print(f"  → GANADOR: NB (ventaja: +{ventaja:.4f})")
                else:
                    ventaja = 0
                    ganador = "Empate"
                    print(f"  → EMPATE")
                
                ventajas_detalladas[categoria] = {
                    'ganador': ganador,
                    'ventaja': ventaja,
                    'nb_mejor': nb_data['Mejor F1'],
                    'lr_mejor': lr_data['Mejor F1'],
                    'nb_promedio': nb_data['F1 Promedio'],
                    'lr_promedio': lr_data['F1 Promedio']
                }
    
    return mejor_por_categoria, ventajas_detalladas

def analizar_nb_vs_lr_por_representacion(tablas_categoria, categorias):
    """
    Analiza cómo se comportan NB vs LR con diferentes representaciones
    """
    print("\n" + "="*80)
    print("NB vs LR POR TIPO DE REPRESENTACIÓN")
    print("="*80)
    
    representaciones = ['TF', 'TF-IDF', 'Lexicon']
    
    for repr_type in representaciones:
        print(f"\n{repr_type}:")
        print("-" * 30)
        
        nb_scores = []
        lr_scores = []
        
        for categoria in categorias:
            if categoria in tablas_categoria:
                tabla = tablas_categoria[categoria]
                
                # Buscar modelos correspondientes
                if repr_type == 'TF':
                    nb_model = 'NB + TF'
                    lr_model = 'LR + TF'
                elif repr_type == 'TF-IDF':
                    nb_model = 'NB + TF-IDF'
                    lr_model = 'LR + TF-IDF'
                else:  # Lexicon
                    nb_model = 'NB + Lexicon(SWN)'
                    lr_model = 'LR + Lexicon(SWN)'
                
                if nb_model in tabla.index:
                    nb_scores.append(tabla.loc[nb_model, 'F1 Macro'])
                if lr_model in tabla.index:
                    lr_scores.append(tabla.loc[lr_model, 'F1 Macro'])
        
        if nb_scores and lr_scores:
            nb_promedio = np.mean(nb_scores)
            lr_promedio = np.mean(lr_scores)
            diferencia = lr_promedio - nb_promedio
            
            print(f"  NB promedio:  {nb_promedio:.4f}")
            print(f"  LR promedio:  {lr_promedio:.4f}")
            print(f"  Diferencia:   {diferencia:+.4f}")
            
            if diferencia > 0.01:
                print(f"  → LR claramente superior con {repr_type}")
            elif diferencia < -0.01:
                print(f"  → NB claramente superior con {repr_type}")
            else:
                print(f"  → Rendimiento similar con {repr_type}")

def crear_estadisticas_detalladas_nb_lr(tablas_categoria, categorias):
    """
    Crea estadísticas detalladas para el análisis NB vs LR
    """
    print("\n" + "="*80)
    print("ESTADÍSTICAS DETALLADAS: NB vs LR")
    print("="*80)
    
    # Recopilar todas las métricas
    todas_metricas = {
        'NB': {'F1': [], 'Accuracy': [], 'Precision': [], 'Recall': []},
        'LR': {'F1': [], 'Accuracy': [], 'Precision': [], 'Recall': []}
    }
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            for modelo in tabla.index:
                if 'NB' in modelo:
                    todas_metricas['NB']['F1'].append(tabla.loc[modelo, 'F1 Macro'])
                    todas_metricas['NB']['Accuracy'].append(tabla.loc[modelo, 'Accuracy'])
                    todas_metricas['NB']['Precision'].append(tabla.loc[modelo, 'Precision Macro'])
                    todas_metricas['NB']['Recall'].append(tabla.loc[modelo, 'Recall Macro'])
                elif 'LR' in modelo:
                    todas_metricas['LR']['F1'].append(tabla.loc[modelo, 'F1 Macro'])
                    todas_metricas['LR']['Accuracy'].append(tabla.loc[modelo, 'Accuracy'])
                    todas_metricas['LR']['Precision'].append(tabla.loc[modelo, 'Precision Macro'])
                    todas_metricas['LR']['Recall'].append(tabla.loc[modelo, 'Recall Macro'])
    
    # Análisis estadístico
    print("\nCOMPARACIÓN ESTADÍSTICA:")
    print("-" * 40)
    
    metricas_nombres = ['F1', 'Accuracy', 'Precision', 'Recall']
    
    for metrica in metricas_nombres:
        nb_vals = todas_metricas['NB'][metrica]
        lr_vals = todas_metricas['LR'][metrica]
        
        if nb_vals and lr_vals:
            print(f"\n{metrica}:")
            print(f"  NB: μ={np.mean(nb_vals):.4f}, σ={np.std(nb_vals):.4f}, min={np.min(nb_vals):.4f}, max={np.max(nb_vals):.4f}")
            print(f"  LR: μ={np.mean(lr_vals):.4f}, σ={np.std(lr_vals):.4f}, min={np.min(lr_vals):.4f}, max={np.max(lr_vals):.4f}")
            
            diferencia = np.mean(lr_vals) - np.mean(nb_vals)
            print(f"  Diferencia (LR-NB): {diferencia:+.4f}")
    
    # Contar victorias
    print(f"\nCONTEO DE VICTORIAS:")
    print("-" * 25)
    
    victorias = {'NB': 0, 'LR': 0, 'Empates': 0}
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            nb_models = [modelo for modelo in tabla.index if 'NB' in modelo]
            lr_models = [modelo for modelo in tabla.index if 'LR' in modelo]
            
            if nb_models and lr_models:
                best_nb = tabla.loc[nb_models, 'F1 Macro'].max()
                best_lr = tabla.loc[lr_models, 'F1 Macro'].max()
                
                if best_lr > best_nb:
                    victorias['LR'] += 1
                elif best_nb > best_lr:
                    victorias['NB'] += 1
                else:
                    victorias['Empates'] += 1
    
    total_comparaciones = sum(victorias.values())
    for algoritmo, wins in victorias.items():
        porcentaje = (wins / total_comparaciones) * 100 if total_comparaciones > 0 else 0
        print(f"  {algoritmo}: {wins}/{total_comparaciones} ({porcentaje:.1f}%)")
    
    return todas_metricas, victorias

# Función principal para ejecutar todas las comparaciones NB vs LR
def ejecutar_comparacion_nb_lr_completa(tablas_categoria, categorias):
    """
    Ejecuta todas las comparaciones entre Naive Bayes y Logistic Regression
    """
    # 1. Comparación general
    df_algoritmos = comparar_nb_vs_lr(tablas_categoria, categorias)
    
    # 2. Análisis por categoría
    mejor_por_cat, ventajas = analizar_nb_vs_lr_por_categoria(tablas_categoria, categorias)
    
    # 3. Análisis por representación
    analizar_nb_vs_lr_por_representacion(tablas_categoria, categorias)
    
    # 4. Estadísticas detalladas
    metricas, victorias = crear_estadisticas_detalladas_nb_lr(tablas_categoria, categorias)
    
    return df_algoritmos, mejor_por_cat, ventajas, metricas, victorias

# Ejemplo de uso:
df_alg, mejor_cat, ventajas, metricas, victorias = ejecutar_comparacion_nb_lr_completa(tablas_categoria, CATEGORIES)

COMPARACIÓN DE ALGORITMOS: NAIVE BAYES vs LOGISTIC REGRESSION

RENDIMIENTO PROMEDIO POR ALGORITMO:
------------------------------------------------------------
                     F1 Macro Promedio  F1 Macro Std  Accuracy Promedio  \
Logistic Regression             0.8000        0.0611             0.8001   
Naive Bayes                     0.7837        0.0788             0.7854   

                     Accuracy Std  
Logistic Regression        0.0610  
Naive Bayes                0.0777  

DIFERENCIA PROMEDIO (LR - NB): +0.0163
→ Logistic Regression supera a Naive Bayes en promedio

ANÁLISIS NB vs LR POR CATEGORÍA

Books:
----------------------------------------
  Naive Bayes:
    Mejor:    F1=0.7836 (NB + TF-IDF)
    Promedio: F1=0.7302
  Logistic Regression:
    Mejor:    F1=0.8065 (LR + TF)
    Promedio: F1=0.7728
  → GANADOR: LR (ventaja: +0.0229)

DVD:
----------------------------------------
  Naive Bayes:
    Mejor:    F1=0.8513 (NB + TF)
    Promedio: F1=0.7843
  Logistic Regre

### Identificación de categorías difíciles

In [25]:
print("="*80)
print("ANÁLISIS DE DIFICULTAD POR CATEGORÍA")
print("="*80)

# Recopilar métricas por categoría
categoria_metrics = {}

for categoria, tabla in tablas_categoria.items():
    if len(tabla) > 0:
        # Obtener la mejor métrica F1 Macro de cada categoría
        best_f1_macro = tabla['F1 Macro'].max()
        best_f1_micro = tabla['F1 Micro'].max()
        best_accuracy = tabla['Accuracy'].max()
        
        # Obtener métricas del mejor modelo
        best_model_idx = tabla['F1 Macro'].idxmax()
        best_model_name = best_model_idx
        
        categoria_metrics[categoria] = {
            'Best F1 Macro': best_f1_macro,
            'Best F1 Micro': best_f1_micro,
            'Best Accuracy': best_accuracy,
            'Best Model': best_model_name
        }

# Crear DataFrame para análisis
df_dificultad = pd.DataFrame(categoria_metrics).T
df_dificultad = df_dificultad.sort_values('Best F1 Macro')

print("\nRANKING DE DIFICULTAD (de más difícil a más fácil):")
print(df_dificultad.round(4))

# Identificar la más difícil y la más fácil
mas_dificil = df_dificultad.index[0]
mas_facil = df_dificultad.index[-1]

print(f"\nCATEGORÍA MÁS DIFÍCIL: {mas_dificil}")
print(f"   F1-Macro: {df_dificultad.loc[mas_dificil, 'Best F1 Macro']:.4f}")
print(f"   Mejor modelo: {df_dificultad.loc[mas_dificil, 'Best Model']}")

print(f"\nCATEGORÍA MÁS FÁCIL: {mas_facil}")
print(f"   F1-Macro: {df_dificultad.loc[mas_facil, 'Best F1 Macro']:.4f}")
print(f"   Mejor modelo: {df_dificultad.loc[mas_facil, 'Best Model']}")

# Diferencias de rendimiento
diferencia = df_dificultad.loc[mas_facil, 'Best F1 Macro'] - df_dificultad.loc[mas_dificil, 'Best F1 Macro']
print(f"\nDIFERENCIA DE RENDIMIENTO: {diferencia:.4f}")

ANÁLISIS DE DIFICULTAD POR CATEGORÍA

RANKING DE DIFICULTAD (de más difícil a más fácil):
            Best F1 Macro Best F1 Micro Best Accuracy   Best Model
Books            0.806493      0.806667      0.806667      LR + TF
Electronics      0.844979         0.845         0.845  NB + TF-IDF
DVD              0.851319      0.851667      0.851667      NB + TF
Kitchen             0.885         0.885         0.885      LR + TF

CATEGORÍA MÁS DIFÍCIL: Books
   F1-Macro: 0.8065
   Mejor modelo: LR + TF

CATEGORÍA MÁS FÁCIL: Kitchen
   F1-Macro: 0.8850
   Mejor modelo: LR + TF

DIFERENCIA DE RENDIMIENTO: 0.0785


### Evaluación de características más importantes por categorías

In [26]:
print("\n" + "="*80)
print("CARACTERÍSTICAS MÁS IMPORTANTES POR CATEGORÍA (LR Parameters)")
print("="*80)

def extraer_top_features_categoria(modelos_categoria, categoria, top_k=15):
    """Extrae top features de LR para una categoría específica"""
    
    if categoria in modelos_categoria:
        models = modelos_categoria[categoria]
        
        # Buscar modelo LR + TF-IDF (suele ser el mejor)
        lr_model = None
        modelo_name = None
        
        for name, model in models.items():
            if 'LR' in name and 'TF-IDF' in name:
                lr_model = model
                modelo_name = name
                break
        
        if lr_model is None:
            # Si no hay TF-IDF, buscar cualquier LR
            for name, model in models.items():
                if 'LR' in name:
                    lr_model = model
                    modelo_name = name
                    break
        
        if lr_model is not None:
            print(f"\nCATEGORÍA: {categoria}")
            print(f"Modelo: {modelo_name}")
            print("-" * 60)
            
            # Extraer vectorizador y coeficientes
            if hasattr(lr_model.named_steps, 'dict'):
                vectorizer = lr_model.named_steps['dict']
                feature_names = vectorizer.get_feature_names_out()
                
                if hasattr(lr_model.named_steps, 'lr'):
                    lr_classifier = lr_model.named_steps['lr']
                    coefs = lr_classifier.coef_.ravel()
                    
                    # Top features positivas (pro-positivas)
                    top_pos_idx = np.argsort(coefs)[-top_k:]
                    # Top features negativas (pro-negativas)  
                    top_neg_idx = np.argsort(coefs)[:top_k]
                    
                    print("\nCARACTERÍSTICAS PRO-POSITIVAS:")
                    for i in reversed(top_pos_idx):
                        print(f"   {feature_names[i]:25s} : {coefs[i]:+8.4f}")
                    
                    print("\nCARACTERÍSTICAS PRO-NEGATIVAS:")
                    for i in top_neg_idx:
                        print(f"   {feature_names[i]:25s} : {coefs[i]:+8.4f}")
                    
                    return {
                        'top_positive': [(feature_names[i], coefs[i]) for i in reversed(top_pos_idx)],
                        'top_negative': [(feature_names[i], coefs[i]) for i in top_neg_idx]
                    }
            else:
                print(f"   No se pudo extraer características - no hay DictVectorizer")
        else:
            print(f"   No se encontró modelo LR para {categoria}")
    else:
        print(f"   Categoría {categoria} no encontrada en modelos")
    
    return None

# Extraer características para cada categoría
CATEGORIAS = ["Books", "DVD", "Electronics", "Kitchen"]
features_por_categoria = {}

for categoria in CATEGORIAS:
    features = extraer_top_features_categoria(modelos_categoria, categoria)
    if features:
        features_por_categoria[categoria] = features

print(f"\n{'='*60}")
print("RESUMEN DE CARACTERÍSTICAS POR CATEGORÍA")
print(f"{'='*60}")

for categoria, features in features_por_categoria.items():
    print(f"\n{categoria}:")
    print("  Top Positivas:", [word for word, coef in features['top_positive'][:5]])
    print("  Top Negativas:", [word for word, coef in features['top_negative'][:5]])



CARACTERÍSTICAS MÁS IMPORTANTES POR CATEGORÍA (LR Parameters)

CATEGORÍA: Books
Modelo: LR + TF-IDF
------------------------------------------------------------

CARACTERÍSTICAS PRO-POSITIVAS:
   great                     :  +1.3768
   excellent                 :  +1.2891
   recommend                 :  +0.8717
   love                      :  +0.8584
   wonderful                 :  +0.8498
   best                      :  +0.8440
   the_best                  :  +0.8184
   easy                      :  +0.7792
   an_excellent              :  +0.7354
   loved                     :  +0.6922
   a_great                   :  +0.6780
   favorite                  :  +0.6761
   highly                    :  +0.6751
   my                        :  +0.6293
   life                      :  +0.6230

CARACTERÍSTICAS PRO-NEGATIVAS:
   not                       :  -1.6153
   i                         :  -1.6071
   no                        :  -1.5530
   bad                       :  -1.2817
   was        

## Evaluación de los modelos globales

A continuación se encuentra la implementación del modelo global que clasifica todas las categorías.

In [20]:
# =============================================================================
# CELDA 6: MODELOS GLOBALES (TODAS LAS CATEGORÍAS COMBINADAS)
# =============================================================================

print(f"\n{'='*100}")
print("ANÁLISIS GLOBAL: TODAS LAS CATEGORÍAS COMBINADAS")
print(f"{'='*100}")

# Cargar datos combinados
print("Cargando datos combinados...")
all_data_combined = load_all_domains_combined(ROOT, CATEGORIES)
X_labeled_global = all_data_combined["X_labeled"]
y_labeled_global = all_data_combined["y_labeled"]
X_unlabeled_global = all_data_combined["X_unlabeled"]

print(f"Datos etiquetados: {len(X_labeled_global)}")
print(f"Datos no etiquetados: {len(X_unlabeled_global)}")


ANÁLISIS GLOBAL: TODAS LAS CATEGORÍAS COMBINADAS
Cargando datos combinados...
Datos etiquetados: 8000
Datos no etiquetados: 19677


In [21]:
X_tr_global, X_te_global, y_tr_global, y_te_global = train_test_split(
    X_labeled_global, y_labeled_global,
    test_size=0.30, random_state=42, stratify=y_labeled_global
)

# Definir modelos globales (mismos 6 tipos)
modelos_globales = {
    "NB + TF (Global)": make_nb_tf(),
    "NB + TF-IDF (Global)": make_nb_tfidf(),
    "LR + TF (Global)": make_lr_tf(),
    "LR + TF-IDF (Global)": make_lr_tfidf(),
    "LR + Lexicon(SWN) (Global)": make_lr_lexicon(lexicon),
    "NB + Lexicon(SWN) (Global)": make_nb_lexicon(lexicon)
}

In [22]:
# Evaluar modelos globales
resultados_globales = {}
modelos_globales_entrenados = {}

for nombre, pipe in modelos_globales.items():
    print(f"\nEntrenando modelo global: {nombre}...")
    try:
        resultado = evaluar_modelo(nombre, pipe, X_tr_global, y_tr_global, X_te_global, y_te_global, mostrar_reporte=True)
        resultados_globales[nombre] = resultado
        modelos_globales_entrenados[nombre] = pipe
    except Exception as e:
        print(f"Error con {nombre}: {e}")
        continue


Entrenando modelo global: NB + TF (Global)...

NB + TF (Global)
Accuracy: 0.8313
Macro  -> P: 0.8342  R: 0.8313  F1: 0.8309
Micro  -> P: 0.8313  R: 0.8313  F1: 0.8313

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.803     0.878     0.839      1200
    positive      0.866     0.784     0.823      1200

    accuracy                          0.831      2400
   macro avg      0.834     0.831     0.831      2400
weighted avg      0.834     0.831     0.831      2400


Entrenando modelo global: NB + TF-IDF (Global)...

NB + TF-IDF (Global)
Accuracy: 0.8433
Macro  -> P: 0.8509  R: 0.8433  F1: 0.8425
Micro  -> P: 0.8433  R: 0.8433  F1: 0.8433

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.799     0.917     0.854      1200
    positive      0.902     0.770     0.831      1200

    accuracy                          0.843      2400
   macro avg      0.851     0.843     0.842      2400
weighted avg     

c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



LR + TF (Global)
Accuracy: 0.8529
Macro  -> P: 0.8530  R: 0.8529  F1: 0.8529
Micro  -> P: 0.8529  R: 0.8529  F1: 0.8529

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.847     0.862     0.854      1200
    positive      0.859     0.844     0.852      1200

    accuracy                          0.853      2400
   macro avg      0.853     0.853     0.853      2400
weighted avg      0.853     0.853     0.853      2400


Entrenando modelo global: LR + TF-IDF (Global)...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



LR + TF-IDF (Global)
Accuracy: 0.8467
Macro  -> P: 0.8476  R: 0.8467  F1: 0.8466
Micro  -> P: 0.8467  R: 0.8467  F1: 0.8467

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.830     0.873     0.851      1200
    positive      0.866     0.821     0.843      1200

    accuracy                          0.847      2400
   macro avg      0.848     0.847     0.847      2400
weighted avg      0.848     0.847     0.847      2400


Entrenando modelo global: LR + Lexicon(SWN) (Global)...


c:\ProgramData\anaconda3\envs\ir-gensim\lib\site-packages\sklearn\linear_model\_logistic.py:1281: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(



LR + Lexicon(SWN) (Global)
Accuracy: 0.7121
Macro  -> P: 0.7127  R: 0.7121  F1: 0.7119
Micro  -> P: 0.7121  R: 0.7121  F1: 0.7121

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.701     0.739     0.720      1200
    positive      0.724     0.685     0.704      1200

    accuracy                          0.712      2400
   macro avg      0.713     0.712     0.712      2400
weighted avg      0.713     0.712     0.712      2400


Entrenando modelo global: NB + Lexicon(SWN) (Global)...

NB + Lexicon(SWN) (Global)
Accuracy: 0.6875
Macro  -> P: 0.6962  R: 0.6875  F1: 0.6840
Micro  -> P: 0.6875  R: 0.6875  F1: 0.6875

Reporte por clase:
              precision    recall  f1-score   support

    negative      0.655     0.792     0.717      1200
    positive      0.737     0.583     0.651      1200

    accuracy                          0.688      2400
   macro avg      0.696     0.688     0.684      2400
weighted avg      0.696     0.688     0.68

In [23]:
df_globales = pd.DataFrame(resultados_globales).T
print(f"\n{'='*80}")
print("RESUMEN MODELOS GLOBALES")
print(f"{'='*80}")
print(df_globales[["Accuracy", "F1 Macro", "F1 Micro"]].round(4))



RESUMEN MODELOS GLOBALES
                            Accuracy  F1 Macro  F1 Micro
NB + TF (Global)              0.8312    0.8309    0.8312
NB + TF-IDF (Global)          0.8433    0.8425    0.8433
LR + TF (Global)              0.8529    0.8529    0.8529
LR + TF-IDF (Global)          0.8467    0.8466    0.8467
LR + Lexicon(SWN) (Global)    0.7121    0.7119    0.7121
NB + Lexicon(SWN) (Global)    0.6875    0.6840    0.6875


In [24]:
# Predicciones en unlabeled global
print(f"\n{'='*80}")
print("PREDICCIONES EN DATOS UNLABELED GLOBALES")
print(f"{'='*80}")

for nombre, pipe in modelos_globales_entrenados.items():
    y_pred_unl_global = pipe.predict(X_unlabeled_global)
    print(f"\n{nombre}")
    print(f"Predicciones (primeros 20): {list(y_pred_unl_global[:20])}")
    print(f"Distribución: Pos={np.sum(y_pred_unl_global)} | Neg={len(y_pred_unl_global)-np.sum(y_pred_unl_global)}")



PREDICCIONES EN DATOS UNLABELED GLOBALES

NB + TF (Global)
Predicciones (primeros 20): [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1]
Distribución: Pos=8973 | Neg=10704

NB + TF-IDF (Global)
Predicciones (primeros 20): [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
Distribución: Pos=8671 | Neg=11006

LR + TF (Global)
Predicciones (primeros 20): [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
Distribución: Pos=9872 | Neg=9805

LR + TF-IDF (Global)
Predicciones (primeros 20): [0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1]
Distribución: Pos=9544 | Neg=10133

LR + Lexicon(SWN) (Global)
Predicciones (primeros 20): [0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
Distribución: Pos=9689 | Neg=9988

NB + Lexicon(SWN) (Global)
Predicciones (primeros 20): [0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1]
Distribución: Pos=7974 | Neg=11703


In [31]:
# =============================================================================
# COMPARACIÓN DE MODELOS GLOBALES POR TIPO DE REPRESENTACIÓN DE CARACTERÍSTICAS
# =============================================================================

def comparar_por_tipo_representacion_global(df_globales):
    """
    Compara el rendimiento de modelos globales por tipo de representación
    """
    print("="*80)
    print("COMPARACIÓN GLOBAL POR TIPO DE REPRESENTACIÓN DE CARACTERÍSTICAS")
    print("="*80)
    
    # Agrupar modelos por tipo de representación
    tipos_representacion = {
        'TF (Term Frequency)': ['NB + TF (Global)', 'LR + TF (Global)'],
        'TF-IDF': ['NB + TF-IDF (Global)', 'LR + TF-IDF (Global)'], 
        'Lexicon Features': ['LR + Lexicon(SWN) (Global)', 'NB + Lexicon(SWN) (Global)']
    }
    
    resultados_por_tipo = {}
    
    for tipo, modelos in tipos_representacion.items():
        f1_scores = []
        accuracy_scores = []
        precision_scores = []
        recall_scores = []
        
        for modelo in modelos:
            if modelo in df_globales.index:
                f1_scores.append(df_globales.loc[modelo, 'F1 Macro'])
                accuracy_scores.append(df_globales.loc[modelo, 'Accuracy'])
                precision_scores.append(df_globales.loc[modelo, 'Precision Macro'])
                recall_scores.append(df_globales.loc[modelo, 'Recall Macro'])
        
        if f1_scores:
            resultados_por_tipo[tipo] = {
                'F1 Macro Promedio': np.mean(f1_scores),
                'F1 Macro Std': np.std(f1_scores),
                'Accuracy Promedio': np.mean(accuracy_scores),
                'Accuracy Std': np.std(accuracy_scores),
                'Precision Promedio': np.mean(precision_scores),
                'Recall Promedio': np.mean(recall_scores),
                'Mejor F1': np.max(f1_scores),
                'Peor F1': np.min(f1_scores),
                'Modelos': modelos,
                'Scores': f1_scores
            }
    
    # Crear DataFrame para comparación
    df_tipos = pd.DataFrame(resultados_por_tipo).T
    df_tipos = df_tipos.sort_values('F1 Macro Promedio', ascending=False)
    
    print("\nRENDIMIENTO PROMEDIO POR TIPO DE REPRESENTACIÓN (MODELOS GLOBALES):")
    print("-" * 70)
    print(df_tipos[['F1 Macro Promedio', 'F1 Macro Std', 'Accuracy Promedio', 'Precision Promedio', 'Recall Promedio']].round(4))
    
    # Análisis de varianza entre tipos
    print("\n\nANÁLISIS DE VARIANZA ENTRE REPRESENTACIONES:")
    print("-" * 50)
    varianzas = df_tipos['F1 Macro Std'].sort_values()
    print("Representación más consistente (menor std):")
    print(varianzas)
    
    # Mejores modelos por tipo
    print("\n\nMEJORES MODELOS POR TIPO DE REPRESENTACIÓN:")
    print("-" * 50)
    for tipo, data in resultados_por_tipo.items():
        mejor_idx = np.argmax(data['Scores'])
        mejor_modelo = data['Modelos'][mejor_idx]
        mejor_score = data['Scores'][mejor_idx]
        print(f"{tipo:25s}: {mejor_modelo:30s} (F1={mejor_score:.4f})")
    
    return df_tipos, resultados_por_tipo

def analizar_complementariedad_global(df_globales):
    """
    Analiza complementariedad entre representaciones en modelos globales
    """
    print("\n" + "="*80)
    print("ANÁLISIS DE COMPLEMENTARIEDAD ENTRE REPRESENTACIONES (GLOBAL)")
    print("="*80)
    
    # Extraer scores por tipo de representación
    tf_scores = []
    tfidf_scores = []
    lexicon_scores = []
    
    modelos_tf = [m for m in df_globales.index if 'TF (Global)' in m]
    modelos_tfidf = [m for m in df_globales.index if 'TF-IDF (Global)' in m]
    modelos_lexicon = [m for m in df_globales.index if 'Lexicon(SWN) (Global)' in m]
    
    if modelos_tf:
        tf_scores = df_globales.loc[modelos_tf, 'F1 Macro'].tolist()
    if modelos_tfidf:
        tfidf_scores = df_globales.loc[modelos_tfidf, 'F1 Macro'].tolist()
    if modelos_lexicon:
        lexicon_scores = df_globales.loc[modelos_lexicon, 'F1 Macro'].tolist()
    
    print("\nCOMPARACIÓN DIRECTA DE RENDIMIENTOS:")
    print("-" * 40)
    
    if tf_scores and tfidf_scores:
        tf_best = max(tf_scores)
        tfidf_best = max(tfidf_scores)
        tf_avg = np.mean(tf_scores)
        tfidf_avg = np.mean(tfidf_scores)
        
        print(f"TF      → Mejor: {tf_best:.4f}, Promedio: {tf_avg:.4f}")
        print(f"TF-IDF  → Mejor: {tfidf_best:.4f}, Promedio: {tfidf_avg:.4f}")
        print(f"Mejora TF-IDF vs TF: {tfidf_best - tf_best:+.4f} (mejor), {tfidf_avg - tf_avg:+.4f} (promedio)")
    
    if lexicon_scores:
        lexicon_best = max(lexicon_scores)
        lexicon_avg = np.mean(lexicon_scores)
        print(f"Lexicon → Mejor: {lexicon_best:.4f}, Promedio: {lexicon_avg:.4f}")
        
        if tfidf_scores:
            print(f"Diferencia Lexicon vs TF-IDF: {lexicon_best - tfidf_best:+.4f} (mejor), {lexicon_avg - tfidf_avg:+.4f} (promedio)")
    
    print("\nINTERPRETACIÓN PARA MODELOS GLOBALES:")
    print("-" * 40)
    print("• TF-IDF vs TF: Normalización crucial para datos multi-dominio")
    print("• Lexicon: Características semánticas vs estadísticas textuales")
    print("• Modelo global: Mayor diversidad de datos puede favorecer representaciones robustas")
    
    return {
        'tf_scores': tf_scores,
        'tfidf_scores': tfidf_scores, 
        'lexicon_scores': lexicon_scores
    }

def comparar_mejor_representacion_global(df_globales):
    """
    Identifica la mejor representación para el modelo global
    """
    print("\n" + "="*80)
    print("MEJOR REPRESENTACIÓN PARA MODELO GLOBAL")
    print("="*80)
    
    # Agrupar por tipo de representación
    tf_models = [modelo for modelo in df_globales.index if 'TF (Global)' in modelo]
    tfidf_models = [modelo for modelo in df_globales.index if 'TF-IDF (Global)' in modelo]
    lexicon_models = [modelo for modelo in df_globales.index if 'Lexicon(SWN) (Global)' in modelo]
    
    resultados_globales = {}
    
    # Mejor modelo por tipo
    if tf_models:
        best_tf = df_globales.loc[tf_models, 'F1 Macro'].max()
        best_tf_model = df_globales.loc[tf_models, 'F1 Macro'].idxmax()
        resultados_globales['TF'] = {'F1': best_tf, 'Modelo': best_tf_model}
    
    if tfidf_models:
        best_tfidf = df_globales.loc[tfidf_models, 'F1 Macro'].max()
        best_tfidf_model = df_globales.loc[tfidf_models, 'F1 Macro'].idxmax()
        resultados_globales['TF-IDF'] = {'F1': best_tfidf, 'Modelo': best_tfidf_model}
    
    if lexicon_models:
        best_lexicon = df_globales.loc[lexicon_models, 'F1 Macro'].max()
        best_lexicon_model = df_globales.loc[lexicon_models, 'F1 Macro'].idxmax()
        resultados_globales['Lexicon'] = {'F1': best_lexicon, 'Modelo': best_lexicon_model}
    
    print("\nRENDIMIENTO POR TIPO DE REPRESENTACIÓN:")
    print("-" * 50)
    for repr_type, data in resultados_globales.items():
        print(f"  {repr_type:12s}: F1={data['F1']:.4f} ({data['Modelo']})")
    
    # Identificar la mejor representación global
    if resultados_globales:
        mejor_repr = max(resultados_globales.keys(), 
                       key=lambda x: resultados_globales[x]['F1'])
        mejor_score = resultados_globales[mejor_repr]['F1']
        mejor_modelo = resultados_globales[mejor_repr]['Modelo']
        
        print(f"\n→ MEJOR REPRESENTACIÓN GLOBAL: {mejor_repr}")
        print(f"  Modelo: {mejor_modelo}")
        print(f"  F1 Score: {mejor_score:.4f}")
        
        # Calcular ventajas
        print(f"\nVENTAJAS SOBRE OTRAS REPRESENTACIONES:")
        for repr_type, data in resultados_globales.items():
            if repr_type != mejor_repr:
                ventaja = mejor_score - data['F1']
                print(f"  vs {repr_type}: +{ventaja:.4f}")
    
    return resultados_globales

def analizar_impacto_combinacion_dominios(df_globales, tablas_categoria, categorias):
    """
    Analiza cómo la combinación de dominios afecta diferentes representaciones
    """
    print("\n" + "="*80)
    print("IMPACTO DE COMBINAR DOMINIOS EN REPRESENTACIONES")
    print("="*80)
    
    # Calcular promedio de modelos individuales por tipo de representación
    tipos_individuales = {
        'TF': [],
        'TF-IDF': [], 
        'Lexicon': []
    }
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            # TF scores
            tf_models = [m for m in tabla.index if 'TF' in m and 'TF-IDF' not in m]
            if tf_models:
                tipos_individuales['TF'].extend(tabla.loc[tf_models, 'F1 Macro'].tolist())
            
            # TF-IDF scores
            tfidf_models = [m for m in tabla.index if 'TF-IDF' in m]
            if tfidf_models:
                tipos_individuales['TF-IDF'].extend(tabla.loc[tfidf_models, 'F1 Macro'].tolist())
            
            # Lexicon scores
            lexicon_models = [m for m in tabla.index if 'Lexicon' in m]
            if lexicon_models:
                tipos_individuales['Lexicon'].extend(tabla.loc[lexicon_models, 'F1 Macro'].tolist())
    
    # Obtener scores globales
    tipos_globales = {}
    
    tf_global_models = [m for m in df_globales.index if 'TF (Global)' in m]
    if tf_global_models:
        tipos_globales['TF'] = df_globales.loc[tf_global_models, 'F1 Macro'].tolist()
    
    tfidf_global_models = [m for m in df_globales.index if 'TF-IDF (Global)' in m]
    if tfidf_global_models:
        tipos_globales['TF-IDF'] = df_globales.loc[tfidf_global_models, 'F1 Macro'].tolist()
    
    lexicon_global_models = [m for m in df_globales.index if 'Lexicon(SWN) (Global)' in m]
    if lexicon_global_models:
        tipos_globales['Lexicon'] = df_globales.loc[lexicon_global_models, 'F1 Macro'].tolist()
    
    print("\nCOMPARACIÓN: MODELOS INDIVIDUALES vs GLOBALES")
    print("-" * 60)
    
    for tipo in ['TF', 'TF-IDF', 'Lexicon']:
        if tipo in tipos_individuales and tipo in tipos_globales:
            ind_promedio = np.mean(tipos_individuales[tipo])
            ind_mejor = np.max(tipos_individuales[tipo])
            
            glob_promedio = np.mean(tipos_globales[tipo])
            glob_mejor = np.max(tipos_globales[tipo])
            
            print(f"\n{tipo}:")
            print(f"  Individual → Promedio: {ind_promedio:.4f}, Mejor: {ind_mejor:.4f}")
            print(f"  Global     → Promedio: {glob_promedio:.4f}, Mejor: {glob_mejor:.4f}")
            print(f"  Diferencia → Promedio: {glob_promedio - ind_promedio:+.4f}, Mejor: {glob_mejor - ind_mejor:+.4f}")
            
            if glob_promedio > ind_promedio:
                print(f"  → Modelo global MEJOR con {tipo}")
            elif glob_promedio < ind_promedio:
                print(f"  → Modelos individuales MEJORES con {tipo}")
            else:
                print(f"  → Rendimiento similar con {tipo}")
    
    print(f"\nCONCLUSIONES SOBRE COMBINACIÓN DE DOMINIOS:")
    print("-" * 50)
    print("• Modelos globales pueden generalizar mejor entre dominios")
    print("• Representaciones robustas (TF-IDF) se benefician más de más datos")
    print("• Características lexicon pueden ser más estables entre dominios")

# Función principal para ejecutar comparación global de representaciones
def ejecutar_comparacion_representaciones_global(df_globales, tablas_categoria=None, categorias=None):
    """
    Ejecuta todas las comparaciones de representación para modelos globales
    """
    # 1. Comparación general por tipo
    df_tipos, resultados_tipo = comparar_por_tipo_representacion_global(df_globales)
    
    # 2. Mejor representación global
    resultados_globales = comparar_mejor_representacion_global(df_globales)
    
    # 3. Análisis de complementariedad
    scores_repr = analizar_complementariedad_global(df_globales)
    
    # 4. Impacto de combinación (si se proporcionan datos individuales)
    if tablas_categoria is not None and categorias is not None:
        analizar_impacto_combinacion_dominios(df_globales, tablas_categoria, categorias)
    
    return df_tipos, resultados_tipo, resultados_globales, scores_repr

# Ejemplo de uso:
df_tipos, resultados_tipo, resultados_globales, scores_repr = ejecutar_comparacion_representaciones_global(df_globales, tablas_categoria, CATEGORIES)

COMPARACIÓN GLOBAL POR TIPO DE REPRESENTACIÓN DE CARACTERÍSTICAS

RENDIMIENTO PROMEDIO POR TIPO DE REPRESENTACIÓN (MODELOS GLOBALES):
----------------------------------------------------------------------
                    F1 Macro Promedio F1 Macro Std Accuracy Promedio  \
TF-IDF                       0.844525     0.002039             0.845   
TF (Term Frequency)           0.84189     0.011015          0.842083   
Lexicon Features             0.697944     0.013928          0.699792   

                    Precision Promedio Recall Promedio  
TF-IDF                        0.849238           0.845  
TF (Term Frequency)           0.843619        0.842083  
Lexicon Features              0.704429        0.699792  


ANÁLISIS DE VARIANZA ENTRE REPRESENTACIONES:
--------------------------------------------------
Representación más consistente (menor std):
TF-IDF                 0.002039
TF (Term Frequency)    0.011015
Lexicon Features       0.013928
Name: F1 Macro Std, dtype: object


MEJO

In [33]:
# =============================================================================
# COMPARACIÓN DE ALGORITMOS GLOBALES: NAIVE BAYES vs LOGISTIC REGRESSION
# =============================================================================

def comparar_nb_vs_lr_global(df_globales):
    """
    Compara el rendimiento entre Naive Bayes y Logistic Regression en modelos globales
    """
    print("="*80)
    print("COMPARACIÓN ALGORITMOS GLOBALES: NAIVE BAYES vs LOGISTIC REGRESSION")
    print("="*80)
    
    # Agrupar modelos por algoritmo
    algoritmos = {
        'Naive Bayes': [modelo for modelo in df_globales.index if 'NB' in modelo],
        'Logistic Regression': [modelo for modelo in df_globales.index if 'LR' in modelo]
    }
    
    resultados_por_algoritmo = {}
    
    for algoritmo, modelos in algoritmos.items():
        if modelos:
            f1_scores = df_globales.loc[modelos, 'F1 Macro'].tolist()
            accuracy_scores = df_globales.loc[modelos, 'Accuracy'].tolist()
            precision_scores = df_globales.loc[modelos, 'Precision Macro'].tolist()
            recall_scores = df_globales.loc[modelos, 'Recall Macro'].tolist()
            
            resultados_por_algoritmo[algoritmo] = {
                'F1 Macro Promedio': np.mean(f1_scores),
                'F1 Macro Std': np.std(f1_scores),
                'Accuracy Promedio': np.mean(accuracy_scores),
                'Accuracy Std': np.std(accuracy_scores),
                'Precision Promedio': np.mean(precision_scores),
                'Recall Promedio': np.mean(recall_scores),
                'Mejor F1': np.max(f1_scores),
                'Peor F1': np.min(f1_scores),
                'Mejor Modelo': df_globales.loc[modelos, 'F1 Macro'].idxmax(),
                'Num Modelos': len(f1_scores),
                'Todos los F1': f1_scores
            }
    
    # Crear DataFrame para comparación
    df_algoritmos = pd.DataFrame(resultados_por_algoritmo).T
    df_algoritmos = df_algoritmos.sort_values('F1 Macro Promedio', ascending=False)
    
    print("\nRENDIMIENTO PROMEDIO POR ALGORITMO (MODELOS GLOBALES):")
    print("-" * 70)
    cols_mostrar = ['F1 Macro Promedio', 'F1 Macro Std', 'Accuracy Promedio', 
                   'Precision Promedio', 'Recall Promedio', 'Mejor F1']
    print(df_algoritmos[cols_mostrar].round(4))
    
    # Análisis de ventaja estadística
    if 'Naive Bayes' in resultados_por_algoritmo and 'Logistic Regression' in resultados_por_algoritmo:
        nb_scores = resultados_por_algoritmo['Naive Bayes']['Todos los F1']
        lr_scores = resultados_por_algoritmo['Logistic Regression']['Todos los F1']
        
        diferencia_promedio = np.mean(lr_scores) - np.mean(nb_scores)
        diferencia_mejor = resultados_por_algoritmo['Logistic Regression']['Mejor F1'] - resultados_por_algoritmo['Naive Bayes']['Mejor F1']
        
        print(f"\nANÁLISIS DE DIFERENCIAS:")
        print("-" * 30)
        print(f"Diferencia promedio (LR - NB): {diferencia_promedio:+.4f}")
        print(f"Diferencia mejor modelo:       {diferencia_mejor:+.4f}")
        
        if diferencia_promedio > 0:
            print("→ Logistic Regression supera a Naive Bayes en modelos globales")
        else:
            print("→ Naive Bayes supera a Logistic Regression en modelos globales")
    
    return df_algoritmos, resultados_por_algoritmo

def analizar_nb_vs_lr_por_representacion_global(df_globales):
    """
    Analiza cómo se comportan NB vs LR con diferentes representaciones en modelos globales
    """
    print("\n" + "="*80)
    print("NB vs LR POR TIPO DE REPRESENTACIÓN (MODELOS GLOBALES)")
    print("="*80)
    
    representaciones = [
        ('TF', 'TF (Global)'),
        ('TF-IDF', 'TF-IDF (Global)'),
        ('Lexicon', 'Lexicon(SWN) (Global)')
    ]
    
    comparaciones_repr = {}
    
    for repr_name, repr_suffix in representaciones:
        print(f"\n{repr_name}:")
        print("-" * 40)
        
        # Buscar modelos correspondientes
        nb_model = f'NB + {repr_suffix}'
        lr_model = f'LR + {repr_suffix}'
        
        nb_score = None
        lr_score = None
        
        if nb_model in df_globales.index:
            nb_score = df_globales.loc[nb_model, 'F1 Macro']
            nb_acc = df_globales.loc[nb_model, 'Accuracy']
            
        if lr_model in df_globales.index:
            lr_score = df_globales.loc[lr_model, 'F1 Macro']
            lr_acc = df_globales.loc[lr_model, 'Accuracy']
        
        if nb_score is not None and lr_score is not None:
            diferencia_f1 = lr_score - nb_score
            diferencia_acc = lr_acc - nb_acc
            
            print(f"  NB:  F1={nb_score:.4f}, Acc={nb_acc:.4f}")
            print(f"  LR:  F1={lr_score:.4f}, Acc={lr_acc:.4f}")
            print(f"  Diferencia F1:  {diferencia_f1:+.4f}")
            print(f"  Diferencia Acc: {diferencia_acc:+.4f}")
            
            if diferencia_f1 > 0.01:
                print(f"  → LR claramente superior con {repr_name}")
                ganador = "LR"
            elif diferencia_f1 < -0.01:
                print(f"  → NB claramente superior con {repr_name}")
                ganador = "NB"
            else:
                print(f"  → Rendimiento similar con {repr_name}")
                ganador = "Similar"
            
            comparaciones_repr[repr_name] = {
                'nb_f1': nb_score,
                'lr_f1': lr_score,
                'nb_acc': nb_acc,
                'lr_acc': lr_acc,
                'diferencia_f1': diferencia_f1,
                'diferencia_acc': diferencia_acc,
                'ganador': ganador
            }
        else:
            print(f"  → Datos incompletos para {repr_name}")
            comparaciones_repr[repr_name] = None
    
    return comparaciones_repr

def crear_resumen_victorias_global(df_globales):
    """
    Crea resumen de victorias para modelos globales
    """
    print("\n" + "="*80)
    print("RESUMEN DE VICTORIAS: NB vs LR (MODELOS GLOBALES)")
    print("="*80)
    
    representaciones = [
        ('TF', 'TF (Global)'),
        ('TF-IDF', 'TF-IDF (Global)'),
        ('Lexicon', 'Lexicon(SWN) (Global)')
    ]
    
    victorias = {'NB': 0, 'LR': 0, 'Empates': 0}
    detalles_victorias = []
    
    print("\nRESULTADOS POR REPRESENTACIÓN:")
    print("-" * 50)
    
    for repr_name, repr_suffix in representaciones:
        nb_model = f'NB + {repr_suffix}'
        lr_model = f'LR + {repr_suffix}'
        
        if nb_model in df_globales.index and lr_model in df_globales.index:
            nb_score = df_globales.loc[nb_model, 'F1 Macro']
            lr_score = df_globales.loc[lr_model, 'F1 Macro']
            
            if lr_score > nb_score:
                ganador = 'LR'
                victorias['LR'] += 1
                diferencia = lr_score - nb_score
            elif nb_score > lr_score:
                ganador = 'NB'
                victorias['NB'] += 1
                diferencia = nb_score - lr_score
            else:
                ganador = 'Empate'
                victorias['Empates'] += 1
                diferencia = 0
            
            print(f"  {repr_name:12s}: {ganador} gana (diferencia: {diferencia:+.4f})")
            detalles_victorias.append({
                'representacion': repr_name,
                'ganador': ganador,
                'nb_score': nb_score,
                'lr_score': lr_score,
                'diferencia': diferencia
            })
    
    total_comparaciones = sum(victorias.values())
    
    print(f"\nCONTEO TOTAL DE VICTORIAS:")
    print("-" * 30)
    for algoritmo, wins in victorias.items():
        if total_comparaciones > 0:
            porcentaje = (wins / total_comparaciones) * 100
            print(f"  {algoritmo:20s}: {wins}/{total_comparaciones} ({porcentaje:.1f}%)")
    
    return victorias, detalles_victorias

def analizar_estabilidad_algoritmos_global(df_globales):
    """
    Analiza la estabilidad de cada algoritmo en modelos globales
    """
    print("\n" + "="*80)
    print("ANÁLISIS DE ESTABILIDAD: ALGORITMOS GLOBALES")
    print("="*80)
    
    nb_models = [modelo for modelo in df_globales.index if 'NB' in modelo]
    lr_models = [modelo for modelo in df_globales.index if 'LR' in modelo]
    
    nb_f1_scores = df_globales.loc[nb_models, 'F1 Macro'].tolist() if nb_models else []
    lr_f1_scores = df_globales.loc[lr_models, 'F1 Macro'].tolist() if lr_models else []
    
    print("\nESTADÍSTICAS DE ESTABILIDAD:")
    print("-" * 40)
    
    if nb_f1_scores:
        nb_mean = np.mean(nb_f1_scores)
        nb_std = np.std(nb_f1_scores)
        nb_range = max(nb_f1_scores) - min(nb_f1_scores)
        
        print(f"Naive Bayes:")
        print(f"  Promedio: {nb_mean:.4f}")
        print(f"  Std Dev:  {nb_std:.4f}")
        print(f"  Rango:    {nb_range:.4f}")
        print(f"  Scores:   {[f'{s:.3f}' for s in nb_f1_scores]}")
    
    if lr_f1_scores:
        lr_mean = np.mean(lr_f1_scores)
        lr_std = np.std(lr_f1_scores)
        lr_range = max(lr_f1_scores) - min(lr_f1_scores)
        
        print(f"\nLogistic Regression:")
        print(f"  Promedio: {lr_mean:.4f}")
        print(f"  Std Dev:  {lr_std:.4f}")
        print(f"  Rango:    {lr_range:.4f}")
        print(f"  Scores:   {[f'{s:.3f}' for s in lr_f1_scores]}")
    
    # Análisis de estabilidad
    if nb_f1_scores and lr_f1_scores:
        print(f"\nCOMPARACIÓN DE ESTABILIDAD:")
        print("-" * 30)
        
        if nb_std < lr_std:
            print(f"→ NB es más estable (menor varianza)")
            mas_estable = "NB"
        elif lr_std < nb_std:
            print(f"→ LR es más estable (menor varianza)")
            mas_estable = "LR"
        else:
            print(f"→ Estabilidad similar")
            mas_estable = "Similar"
        
        diferencia_estabilidad = abs(nb_std - lr_std)
        print(f"→ Diferencia en std: {diferencia_estabilidad:.4f}")
        
        return {
            'nb_stats': {'mean': nb_mean, 'std': nb_std, 'range': nb_range, 'scores': nb_f1_scores},
            'lr_stats': {'mean': lr_mean, 'std': lr_std, 'range': lr_range, 'scores': lr_f1_scores},
            'mas_estable': mas_estable,
            'diferencia_estabilidad': diferencia_estabilidad
        }
    
    return None

def comparar_global_vs_individual(df_globales, tablas_categoria, categorias):
    """
    Compara rendimiento de modelos globales vs promedio de modelos individuales
    """
    print("\n" + "="*80)
    print("COMPARACIÓN: MODELOS GLOBALES vs INDIVIDUALES")
    print("="*80)
    
    # Calcular promedios de modelos individuales
    nb_individual = []
    lr_individual = []
    
    for categoria in categorias:
        if categoria in tablas_categoria:
            tabla = tablas_categoria[categoria]
            
            for modelo in tabla.index:
                if 'NB' in modelo:
                    nb_individual.append(tabla.loc[modelo, 'F1 Macro'])
                elif 'LR' in modelo:
                    lr_individual.append(tabla.loc[modelo, 'F1 Macro'])
    
    # Obtener scores globales
    nb_global = [df_globales.loc[m, 'F1 Macro'] for m in df_globales.index if 'NB' in m]
    lr_global = [df_globales.loc[m, 'F1 Macro'] for m in df_globales.index if 'LR' in m]
    
    print("\nCOMPARACIÓN DE RENDIMIENTOS:")
    print("-" * 40)
    
    if nb_individual and nb_global:
        nb_ind_promedio = np.mean(nb_individual)
        nb_ind_mejor = np.max(nb_individual)
        nb_glob_promedio = np.mean(nb_global)
        nb_glob_mejor = np.max(nb_global)
        
        print(f"Naive Bayes:")
        print(f"  Individual → Promedio: {nb_ind_promedio:.4f}, Mejor: {nb_ind_mejor:.4f}")
        print(f"  Global     → Promedio: {nb_glob_promedio:.4f}, Mejor: {nb_glob_mejor:.4f}")
        print(f"  Diferencia → Promedio: {nb_glob_promedio - nb_ind_promedio:+.4f}, Mejor: {nb_glob_mejor - nb_ind_mejor:+.4f}")
    
    if lr_individual and lr_global:
        lr_ind_promedio = np.mean(lr_individual)
        lr_ind_mejor = np.max(lr_individual)
        lr_glob_promedio = np.mean(lr_global)
        lr_glob_mejor = np.max(lr_global)
        
        print(f"\nLogistic Regression:")
        print(f"  Individual → Promedio: {lr_ind_promedio:.4f}, Mejor: {lr_ind_mejor:.4f}")
        print(f"  Global     → Promedio: {lr_glob_promedio:.4f}, Mejor: {lr_glob_mejor:.4f}")
        print(f"  Diferencia → Promedio: {lr_glob_promedio - lr_ind_promedio:+.4f}, Mejor: {lr_glob_mejor - lr_ind_mejor:+.4f}")
    
    print(f"\nCONCLUSIONES:")
    print("-" * 20)
    print("• Modelos globales: Entrenados en datos combinados de todos los dominios")
    print("• Modelos individuales: Especializados en cada categoría")
    print("• Comparación muestra trade-off entre especialización vs generalización")

def crear_estadisticas_detalladas_nb_lr_global(df_globales):
    """
    Crea estadísticas detalladas para el análisis NB vs LR global
    """
    print("\n" + "="*80)
    print("ESTADÍSTICAS DETALLADAS: NB vs LR (MODELOS GLOBALES)")
    print("="*80)
    
    # Recopilar todas las métricas
    nb_models = [modelo for modelo in df_globales.index if 'NB' in modelo]
    lr_models = [modelo for modelo in df_globales.index if 'LR' in modelo]
    
    metricas_nb = {
        'F1': df_globales.loc[nb_models, 'F1 Macro'].tolist() if nb_models else [],
        'Accuracy': df_globales.loc[nb_models, 'Accuracy'].tolist() if nb_models else [],
        'Precision': df_globales.loc[nb_models, 'Precision Macro'].tolist() if nb_models else [],
        'Recall': df_globales.loc[nb_models, 'Recall Macro'].tolist() if nb_models else []
    }
    
    metricas_lr = {
        'F1': df_globales.loc[lr_models, 'F1 Macro'].tolist() if lr_models else [],
        'Accuracy': df_globales.loc[lr_models, 'Accuracy'].tolist() if lr_models else [],
        'Precision': df_globales.loc[lr_models, 'Precision Macro'].tolist() if lr_models else [],
        'Recall': df_globales.loc[lr_models, 'Recall Macro'].tolist() if lr_models else []
    }
    
    print("\nCOMPARACIÓN ESTADÍSTICA DETALLADA:")
    print("-" * 45)
    
    for metrica in ['F1', 'Accuracy', 'Precision', 'Recall']:
        nb_vals = metricas_nb[metrica]
        lr_vals = metricas_lr[metrica]
        
        if nb_vals and lr_vals:
            print(f"\n{metrica}:")
            print(f"  NB: μ={np.mean(nb_vals):.4f}, σ={np.std(nb_vals):.4f}, min={np.min(nb_vals):.4f}, max={np.max(nb_vals):.4f}")
            print(f"  LR: μ={np.mean(lr_vals):.4f}, σ={np.std(lr_vals):.4f}, min={np.min(lr_vals):.4f}, max={np.max(lr_vals):.4f}")
            
            diferencia = np.mean(lr_vals) - np.mean(nb_vals)
            print(f"  Diferencia (LR-NB): {diferencia:+.4f}")
    
    return {'NB': metricas_nb, 'LR': metricas_lr}

# Función principal para ejecutar todas las comparaciones NB vs LR globales
def ejecutar_comparacion_nb_lr_global_completa(df_globales, tablas_categoria=None, categorias=None):
    """
    Ejecuta todas las comparaciones entre Naive Bayes y Logistic Regression para modelos globales
    """
    # 1. Comparación general
    df_algoritmos, resultados_alg = comparar_nb_vs_lr_global(df_globales)
    
    # 2. Análisis por representación
    comparaciones_repr = analizar_nb_vs_lr_por_representacion_global(df_globales)
    
    # 3. Resumen de victorias
    victorias, detalles = crear_resumen_victorias_global(df_globales)
    
    # 4. Análisis de estabilidad
    estabilidad = analizar_estabilidad_algoritmos_global(df_globales)
    
    # 5. Comparación global vs individual (si se proporcionan datos)
    if tablas_categoria is not None and categorias is not None:
        comparar_global_vs_individual(df_globales, tablas_categoria, categorias)
    
    # 6. Estadísticas detalladas
    metricas_detalladas = crear_estadisticas_detalladas_nb_lr_global(df_globales)
    
    return df_algoritmos, resultados_alg, comparaciones_repr, victorias, detalles, estabilidad, metricas_detalladas

# Ejemplo de uso:
df_alg, res_alg, comp_repr, victorias, detalles, estabilidad, metricas = ejecutar_comparacion_nb_lr_global_completa(df_globales, tablas_categoria, CATEGORIES)

COMPARACIÓN ALGORITMOS GLOBALES: NAIVE BAYES vs LOGISTIC REGRESSION

RENDIMIENTO PROMEDIO POR ALGORITMO (MODELOS GLOBALES):
----------------------------------------------------------------------
                    F1 Macro Promedio F1 Macro Std Accuracy Promedio  \
Logistic Regression          0.803781     0.065041          0.803889   
Naive Bayes                  0.785793     0.072123          0.787361   

                    Precision Promedio Recall Promedio  Mejor F1  
Logistic Regression           0.804442        0.803889  0.852905  
Naive Bayes                   0.793748        0.787361  0.842486  

ANÁLISIS DE DIFERENCIAS:
------------------------------
Diferencia promedio (LR - NB): +0.0180
Diferencia mejor modelo:       +0.0104
→ Logistic Regression supera a Naive Bayes en modelos globales

NB vs LR POR TIPO DE REPRESENTACIÓN (MODELOS GLOBALES)

TF:
----------------------------------------
  NB:  F1=0.8309, Acc=0.8313
  LR:  F1=0.8529, Acc=0.8529
  Diferencia F1:  +0.0220
  D

In [27]:
print("\n" + "="*80)
print("CARACTERÍSTICAS MÁS IMPORTANTES - MODELO GLOBAL")
print("="*80)

def extraer_features_modelo_global(modelos_globales_entrenados, top_k=15):
    """Extrae características del mejor modelo global"""
    
    # Buscar el mejor modelo LR global
    lr_global_model = None
    modelo_name = None
    
    # Priorizar LR + TF-IDF
    for name, model in modelos_globales_entrenados.items():
        if 'LR' in name and 'TF-IDF' in name:
            lr_global_model = model
            modelo_name = name
            break
    
    # Si no hay TF-IDF, buscar cualquier LR
    if lr_global_model is None:
        for name, model in modelos_globales_entrenados.items():
            if 'LR' in name:
                lr_global_model = model
                modelo_name = name
                break
    
    if lr_global_model is not None:
        print(f"Analizando modelo global: {modelo_name}")
        print("-" * 60)
        
        # Extraer características
        if hasattr(lr_global_model.named_steps, 'dict'):
            vectorizer = lr_global_model.named_steps['dict']
            feature_names = vectorizer.get_feature_names_out()
            
            if hasattr(lr_global_model.named_steps, 'lr'):
                lr_classifier = lr_global_model.named_steps['lr']
                coefs = lr_classifier.coef_.ravel()
                
                # Top features
                top_pos_idx = np.argsort(coefs)[-top_k:]
                top_neg_idx = np.argsort(coefs)[:top_k]
                
                print("\nCARACTERÍSTICAS PRO-POSITIVAS (Modelo Global):")
                for i in reversed(top_pos_idx):
                    print(f"   {feature_names[i]:25s} : {coefs[i]:+8.4f}")
                
                print("\nCARACTERÍSTICAS PRO-NEGATIVAS (Modelo Global):")
                for i in top_neg_idx:
                    print(f"   {feature_names[i]:25s} : {coefs[i]:+8.4f}")
                
                global_features = {
                    'top_positive': [(feature_names[i], coefs[i]) for i in reversed(top_pos_idx)],
                    'top_negative': [(feature_names[i], coefs[i]) for i in top_neg_idx],
                    'model_name': modelo_name
                }
                
                return global_features
        else:
            print("No se pudo extraer características - no hay DictVectorizer")
    else:
        print("No se encontró modelo LR global")
    
    return None

# Extraer características del modelo global
features_global = extraer_features_modelo_global(modelos_globales_entrenados)

if features_global:
    print(f"\n{'='*60}")
    print("COMPARACIÓN: INDIVIDUAL vs GLOBAL")
    print(f"{'='*60}")
    
    print("\nCaracterísticas globales más importantes:")
    print("  Positivas:", [word for word, coef in features_global['top_positive'][:8]])
    print("  Negativas:", [word for word, coef in features_global['top_negative'][:8]])
    
    # Comparar con características por categoría
    if features_por_categoria:
        print("\nComparación con modelos individuales:")
        
        # Palabras más comunes entre categorías
        all_positive_words = []
        all_negative_words = []
        
        for categoria, features in features_por_categoria.items():
            positive_words = [word for word, coef in features['top_positive'][:10]]
            negative_words = [word for word, coef in features['top_negative'][:10]]
            all_positive_words.extend(positive_words)
            all_negative_words.extend(negative_words)
        
        # Encontrar intersecciones
        global_pos_words = [word for word, coef in features_global['top_positive'][:10]]
        global_neg_words = [word for word, coef in features_global['top_negative'][:10]]
        
        common_positive = set(global_pos_words) & set(all_positive_words)
        common_negative = set(global_neg_words) & set(all_negative_words)
        
        print(f"  Palabras positivas comunes: {list(common_positive)[:8]}")
        print(f"  Palabras negativas comunes: {list(common_negative)[:8]}")



CARACTERÍSTICAS MÁS IMPORTANTES - MODELO GLOBAL
Analizando modelo global: LR + TF-IDF (Global)
------------------------------------------------------------

CARACTERÍSTICAS PRO-POSITIVAS (Modelo Global):
   great                     :  +6.0325
   excellent                 :  +3.5396
   love                      :  +3.2534
   easy                      :  +3.1149
   best                      :  +3.0337
   easy_to                   :  +2.4416
   the_best                  :  +2.3529
   perfect                   :  +2.2856
   price                     :  +2.2393
   well                      :  +2.1104
   a_great                   :  +2.0907
   my                        :  +1.8678
   good                      :  +1.8520
   wonderful                 :  +1.7742
   i_love                    :  +1.7147

CARACTERÍSTICAS PRO-NEGATIVAS (Modelo Global):
   not                       :  -5.4339
   bad                       :  -2.9638
   waste                     :  -2.7388
   was                     